In [3]:
import pandas as pd
import numpy as np
import plotly_express as px

## Mungitura

In [22]:
df = pd.read_csv("new_mungitura.csv",sep=';',decimal=','))

/var/folders/48/tzp62xjn58bb_nw1n30kx0sh0000gn/T/ipykernel_39504/3489801357.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_mungitura.csv")


In [23]:
df.head()

,Ora inizio,Numero animale,Gruppo,MS/MP,Azione,Durata (MM:SS),Produzione (kg),Numero mungitura,OCC(*1000 Cell/ml),Kick-off,...,Capezzoli non trovatiAD,Capezzoli non trovatiPS,Capezzoli non trovatiAS,Kick-offPD,Kick-offAD,Kick-offPS,Kick-offAS,Minuti,Secondi,MinutiTotali
0,31/08/2020 23:59,173.0,VMS 3,MS3,Mungitura,05:29,5.31,1.0,NaN,NaN,...,0,0,0,0,0,0,0,5,29,5.48333
1,31/08/2020 23:57,935.0,VMS 1,MS1,Mungitura,12:09,3.44,1.0,NaN,NaN,...,1,0,0,0,0,0,0,12,9,12.15000
2,31/08/2020 23:55,21.0,Asciutte,MS2,Mungitura,07:14,2.22,1.0,NaN,NaN,...,0,0,0,0,0,0,0,7,14,7.23333
3,31/08/2020 23:54,38.0,Asciutte,MS3,Mungitura,04:48,2.97,3.0,NaN,NaN,...,0,0,0,0,0,0,0,4,48,4.80000
4,31/08/2020 23:53,536.0,Asciutte,MS4,Mungitura,09:57,0.07,1.0,NaN,NaN,...,1,1,1,0,0,0,0,9,57,9.95000


In [5]:
df.drop(columns=["MinutiTotali","SecondiTotali"],inplace=True)

In [6]:
df[['Minuti', 'Secondi']] = df['Durata (MM:SS)'].str.split(':', 1, expand=True)
df['Minuti'] = df['Minuti'].str.strip("0")
df['Secondi'] = df['Secondi'].str.strip("0")

In [7]:
df.loc[df["Minuti"] == ''] = "0"
df.loc[df["Secondi"] == ''] = "0"

In [8]:
df["MinutiTotali"] = df["Minuti"].astype("float") + df["Secondi"].astype("float")/60

In [29]:
mask = df["Ora inizio"] == "MANCANO I DATI DAL 17/04/2021 AL 12/02/2021 E NON SONO RECUPERABILI"
i = df[mask].index
df.drop(i,inplace=True)

In [32]:
df['Data'] = pd.to_datetime(df['Ora inizio']).dt.date
df['Tempo'] = pd.to_datetime(df['Ora inizio']).dt.time
df['Mese'] = pd.to_datetime(df['Ora inizio']).dt.month
df['Anno'] = pd.to_datetime(df['Ora inizio']).dt.year
df['Mese Anno'] = [str(x)[0:7] for x in df['Ora inizio']]

In [33]:
columns = ["Produzione (kg)","Flusso AS","Flusso AD","Flusso PS","Flusso PD",
           "Sangue AS","Sangue AD","Sangue PS","Sangue PD",
           "Conducibilità AS","Conducibilità AD","Conducibilità PS","Conducibilità PD",
           "PicchiFlusso AS","PicchiFlusso AD","PicchiFlusso PS","PicchiFlusso PD",
           'Produzione AS', 'Produzione AD','Produzione PS', 'Produzione PD'
]

In [ ]:
df["Produzione PD"].str.replace(",",".").astype(float)

In [37]:
for column in columns:
    df[column] = df[column].astype(str).str.replace(",",".")

In [38]:
df[column] = df[column].astype(float)

In [51]:
base = "Kick-off"
cats = ["PD","AD","AS","PS","Tutto"]
df[base+"PD"]=0
df[base+"AD"]=0
df[base+"PS"]=0
df[base+"AS"]=0

In [ ]:
col = base
for _, i in df.iterrows():
    if str(i[col]) != "nan":
        print(i[col]+" "+str(_))
        if str(i[col]) == "Tutto":
            print("Tutto found "+str(_))
            df.loc[_,base+"AD"] = 1
            df.loc[_,base+"PD"] = 1
            df.loc[_,base+"AS"] = 1
            df.loc[_,base+"PS"] = 1
        else:
            if str(i[base]).find("AD") != -1:
                print("AD found "+str(_))
                df.loc[_,base+"AD"] = 1
            if str(i[base]).find("PD")!= -1:
                print("PD found "+str(_))
                df.loc[_,base+"PD"] = 1
            if str(i[base]).find("PS")!= -1:
                print("PS found "+str(_))
                df.loc[_,base+"PS"] = 1
            if str(i[base]).find("AS")!= -1:
                print("AS found "+str(_))
                df.loc[_,base+"AD"] = 1

In [10]:
df.to_csv("new_mungitura.csv",index=False, float_format='%g')

## Traffico

In [11]:
df = pd.read_csv("new_traffico_animali.csv")

/var/folders/48/tzp62xjn58bb_nw1n30kx0sh0000gn/T/ipykernel_39504/354719305.py:1: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_traffico_animali.csv")


In [16]:
df.drop(columns=["OreTotali","MinutiTotali"],inplace=True)

In [17]:
col = "Tempo nell'area (hh:mm)"

df[['Ore', 'Minuti']] = df[col].str.split(':', 1, expand=True)
df['Minuti'] = df['Minuti'].str.strip("0")
df['Ore'] = df['Ore'].str.strip("0")

In [19]:
df.loc[df["Minuti"] == ''] = "0"
df.loc[df["Ore"] == ''] = "0"

In [20]:
df["OreTotali"] = df["Ore"].astype("float") + df["Minuti"].astype("float")/60

In [ ]:
df['Data'] = pd.to_datetime(df['Data/Ora']).dt.date

In [ ]:
df['Tempo'] = pd.to_datetime(df['Data/Ora']).dt.time
df['Mese'] = pd.to_datetime(df['Data/Ora']).dt.month
df['Anno'] = pd.to_datetime(df['Data/Ora']).dt.year
df['Mese Anno'] = [str(x)[0:7] for x in df['Data']]

In [ ]:
mask = df["Risultato"].str.contains("kg",na=False)

In [ ]:
df["Risultato"] = np.where(mask,df["Risultato"].str.replace(",","."),0)

In [ ]:
df["Risultato"] = df["Risultato"].astype('float')

In [ ]:
df['Risultato'] = df['Risultato'].str.extract('([0-9][,.]*[0-9]*)')

In [5]:
cats = ["PD","AD","AS","PS","Tutto"]
df["PD"]=0
df["AD"]=0
df["PS"]=0
df["AS"]=0

In [ ]:
col = "Capezzoli non completamente munti"
for _, i in df.iterrows():
    if str(i[col]) != "nan":
        print(i[col]+" "+str(_))
        if str(i[col]) == "Tutto":
            print("Tutto found "+str(_))
            df.loc[_,"AD"] = 1
            df.loc[_,"PD"] = 1
            df.loc[_,"AS"] = 1
            df.loc[_,"PS"] = 1
        else:
            if str(i["Capezzoli non completamente munti"]).find("AD") != -1:
                print("AD found "+str(_))
                df.loc[_,"AD"] = 1
            if str(i["Capezzoli non completamente munti"]).find("PD")!= -1:
                print("PD found "+str(_))
                df.loc[_,"PD"] = 1
            if str(i["Capezzoli non completamente munti"]).find("PS")!= -1:
                print("PS found "+str(_))
                df.loc[_,"PS"] = 1
            if str(i["Capezzoli non completamente munti"]).find("AS")!= -1:
                print("AS found "+str(_))
                df.loc[_,"AD"] = 1

In [21]:
df.to_csv("new_traffico_animali.csv",index=False,float_format='%g')

## Mungitura eventi

In [115]:
mungitura_eventi = pd.read_csv("MungituraEventi.csv")

In [116]:
mungitura_eventi['Data'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.date
mungitura_eventi['Tempo'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.time
mungitura_eventi['Mese'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.month
mungitura_eventi['Anno'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.year

In [117]:
mungitura_eventi['Mese Anno'] = [str(x)[0:7] for x in mungitura_eventi['Data']]

## Istogramma eventi

In [84]:
px.histogram(mungitura_eventi,x="Tipo")

In [85]:
px.pie(mungitura_eventi,names="Tipo")

## Serie temporale x=data y=tipo

In [118]:
mungitura_eventi["id"] = range(mungitura_eventi.shape[0])

In [119]:
mungitura_eventi["count"] = mungitura_eventi.groupby(["Mese Anno"])["id"].transform("count")

In [92]:
px.histogram(mungitura_eventi,x="Mese Anno",color="Tipo")

In [95]:
mungitura_eventi.shape

(10422, 14)

In [97]:
fig = px.line(mungitura_eventi, x="Mese Anno",y="count", color="Tipo")
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

In [121]:
mungitura_eventi.to_csv("new_mungitura_eventi.csv",index=False)